In [1]:
from pathlib import Path
from shutil import copyfile
import pickle

Data manipulation

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.exceptions import NotFittedError

Plotting / Visualisation

In [4]:
import matplotlib.pyplot as plt

In [5]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 6)

Gaussian Process Regression

In [6]:
import gpflow
import tensorflow as tf

In [7]:
from gpflow.utilities import print_summary

In [8]:
gpflow.config.set_default_summary_fmt("notebook")

## Load previously exported data

In [9]:
train_exps = ['Exp1', 'Exp3', 'Exp5', 'Exp6']
test_exps = ['Exp2', 'Exp4', 'Exp7']

In [10]:
t_cols = ['time_h', 'time_m']
w_cols = ['SolRad', 'OutsideTemp']
u_cols = ['SimulatedHeat']
y_cols = ['SimulatedTemp']

In [11]:
t_lags = 4
w_lags = 1
u_lags = 3
y_lags = 3

In [12]:
#dict_cols = pickle.load(open(Path("dict_cols.pkl"), 'rb'))
#dict_cols

In [13]:
dict_cols = {
    't': (t_lags, t_cols),
    'w': (w_lags, w_cols),
    'u': (u_lags, u_cols),
    'y': (y_lags, y_cols)
}

In [14]:
dfs_train = pickle.load(open(Path("dfs_train.pkl"), 'rb'))
dfs_test = pickle.load(open(Path("dfs_test.pkl"), 'rb'))

In [15]:
scaler = pickle.load(open(Path("scaler.pkl"), 'rb'))

In [16]:
def get_scaled_df(df, dict_cols, scaler):
    
    t_list = dict_cols['t'][1]
    w_list = dict_cols['w'][1]
    u_list = dict_cols['u'][1]
    y_list = dict_cols['y'][1]
    
    df_local = df[t_list + w_list + u_list + y_list]
    df_scaled = df_local.to_numpy()
    
    try:
        df_scaled = scaler.transform(df_scaled)
    except NotFittedError:
        df_scaled = scaler.fit_transform(df_scaled)
        
    df_scaled = pd.DataFrame(df_scaled, index = df_local.index, columns = df_local.columns)
    
    return df_scaled

In [17]:
df_train = pd.concat(dfs_train)

In [18]:
np.linalg.cond(df_train.to_numpy())

35185.23586737608

In [19]:
df_train_sc = get_scaled_df(df_train, dict_cols, scaler)

In [20]:
np.linalg.cond(df_train_sc.to_numpy())

7.478002157732377

In [21]:
dfs_train_sc = []
dfs_test_sc = []
for df in dfs_train:
    df_sc = get_scaled_df(df, dict_cols, scaler)
    dfs_train_sc.append(df_sc)
    
for df in dfs_test:
    df_sc = get_scaled_df(df, dict_cols, scaler)
    dfs_test_sc.append(df_sc)

In [22]:
def data_to_gpr(df, dict_cols):
    
    t_list = dict_cols['t'][1]
    w_list = dict_cols['w'][1]
    u_list = dict_cols['u'][1]
    y_list = dict_cols['y'][1]
    
    df_gpr = df[t_list + w_list + u_list + y_list].copy()
    
    for lags, names in dict_cols.values():
        for name in names:
            col_idx = df_gpr.columns.get_loc(name)
            for lag in range(1, lags + 1):
                df_gpr.insert(col_idx + lag, f"{name}_{lag}", df_gpr.loc[:, name].shift(lag))

    df_gpr.dropna(inplace = True)
    
    return df_gpr

In [23]:
#dfs_gpr_train = pickle.load(open(Path("dfs_gpr_train.pkl"), 'rb'))
#dfs_gpr_test = pickle.load(open(Path("dfs_gpr_test.pkl"), 'rb'))

In [24]:
dfs_gpr_train = []
for df_sc in dfs_train_sc:
    dfs_gpr_train.append(data_to_gpr(df_sc, dict_cols))
df_gpr_train = pd.concat(dfs_gpr_train)
df_gpr_train.head()

,time_h,time_h_1,time_h_2,time_h_3,time_h_4,time_m,time_m_1,time_m_2,time_m_3,time_m_4,...,OutsideTemp,OutsideTemp_1,SimulatedHeat,SimulatedHeat_1,SimulatedHeat_2,SimulatedHeat_3,SimulatedTemp,SimulatedTemp_1,SimulatedTemp_2,SimulatedTemp_3
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-06-01 20:20:00+02:00,0.73913,0.73913,0.73913,0.73913,0.73913,-0.272727,-0.454545,-0.636364,-0.818182,-1.000000,...,0.058824,0.058824,-0.904139,-0.580115,-0.580115,-0.580115,-0.179560,-0.132679,-0.094006,-0.076890
2017-06-01 20:25:00+02:00,0.73913,0.73913,0.73913,0.73913,0.73913,-0.090909,-0.272727,-0.454545,-0.636364,-0.818182,...,0.058824,0.058824,-0.904139,-0.904139,-0.580115,-0.580115,-0.208254,-0.179560,-0.132679,-0.094006
2017-06-01 20:30:00+02:00,0.73913,0.73913,0.73913,0.73913,0.73913,0.090909,-0.090909,-0.272727,-0.454545,-0.636364,...,0.058824,0.058824,-0.904139,-0.904139,-0.904139,-0.580115,-0.222268,-0.208254,-0.179560,-0.132679
2017-06-01 20:35:00+02:00,0.73913,0.73913,0.73913,0.73913,0.73913,0.272727,0.090909,-0.090909,-0.272727,-0.454545,...,0.058824,0.058824,-0.904139,-0.904139,-0.904139,-0.904139,-0.234855,-0.222268,-0.208254,-0.179560
2017-06-01 20:40:00+02:00,0.73913,0.73913,0.73913,0.73913,0.73913,0.454545,0.272727,0.090909,-0.090909,-0.272727,...,0.058824,0.058824,-0.904139,-0.904139,-0.904139,-0.904139,-0.247166,-0.234855,-0.222268,-0.208254


In [25]:
#df_gpr_train = pd.concat(dfs_gpr_train)

df_input_train = df_gpr_train.drop(columns = dict_cols['u'][1] + dict_cols['y'][1])
df_output_train = df_gpr_train[dict_cols['y'][1]]

np_input_train = df_input_train.to_numpy()
np_output_train = df_output_train.to_numpy().reshape(-1, 1)

In [26]:
## Define Kernel

In [27]:
nb_dims = np_input_train.shape[1]
rational_dims = np.arange(0, (dict_cols['t'][0] + 1) * len(dict_cols['t'][1]), 1)
nb_rational_dims = len(rational_dims)
squared_dims = np.arange(nb_rational_dims, nb_dims, 1)
nb_squared_dims = len(squared_dims)

In [28]:
print(f"rational dims: {nb_rational_dims}")
print(f"squared dims: {nb_squared_dims}")

rational dims: 10
squared dims: 10


In [29]:
squared_l = [1e-4] * nb_squared_dims
rational_l = [1e-7] * nb_rational_dims

In [30]:
squared_l = np.linspace(0.01, 1, nb_squared_dims)
rational_l = np.linspace(0.01, 1, nb_rational_dims)

In [31]:
k0 = gpflow.kernels.SquaredExponential(lengthscales = squared_l, active_dims = squared_dims, variance = 2)
k1 = gpflow.kernels.Constant()
k2 = gpflow.kernels.RationalQuadratic(lengthscales = rational_l, active_dims = rational_dims, variance = 2)
k3 = gpflow.kernels.Periodic(k2)

In [32]:
k = (k0 + k1) * k2
print_summary(k)

name,class,transform,prior,trainable,shape,dtype,value
Product.kernels[0].kernels[0].variance,Parameter,Softplus,,True,(),float64,2.0
Product.kernels[0].kernels[0].lengthscales,Parameter,Softplus,,True,"(10,)",float64,"[0.01, 0.12, 0.23..."
Product.kernels[0].kernels[1].variance,Parameter,Softplus,,True,(),float64,1.0
Product.kernels[1].variance,Parameter,Softplus,,True,(),float64,2.0
Product.kernels[1].lengthscales,Parameter,Softplus,,True,"(10,)",float64,"[0.01, 0.12, 0.23..."
Product.kernels[1].alpha,Parameter,Softplus,,True,(),float64,1.0


## Compile Model

In [33]:
m = gpflow.models.GPR(
    data = (np_input_train, np_output_train), 
    kernel = k, 
    mean_function = None
    )
print_summary(m)

name,class,transform,prior,trainable,shape,dtype,value
GPR.kernel.kernels[0].kernels[0].variance,Parameter,Softplus,,True,(),float64,2.0
GPR.kernel.kernels[0].kernels[0].lengthscales,Parameter,Softplus,,True,"(10,)",float64,"[0.01, 0.12, 0.23..."
GPR.kernel.kernels[0].kernels[1].variance,Parameter,Softplus,,True,(),float64,1.0
GPR.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,2.0
GPR.kernel.kernels[1].lengthscales,Parameter,Softplus,,True,"(10,)",float64,"[0.01, 0.12, 0.23..."
GPR.kernel.kernels[1].alpha,Parameter,Softplus,,True,(),float64,1.0
GPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0


## Train Model

In [34]:
opt = gpflow.optimizers.Scipy()

In [35]:
from datetime import datetime

In [36]:
start_time = datetime.now()
opt.minimize(m.training_loss, m.trainable_variables)
print(f"Finished fitting in {datetime.now() - start_time}")
print_summary(m)

InvalidArgumentError:  Input matrix is not invertible.
	 [[node gradient_tape/triangular_solve/MatrixTriangularSolve (defined at /usr/lib/python3.9/site-packages/gpflow/optimizers/scipy.py:173) ]] [Op:__inference__tf_eval_1118]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/triangular_solve/MatrixTriangularSolve:
 Cholesky (defined at /usr/lib/python3.9/site-packages/gpflow/models/gpr.py:87)

Function call stack:
_tf_eval


## Evaluate performance on training data

In [ ]:
nb_plts = len(dfs_gpr_train)

In [ ]:
plt.figure(figsize = (20, 20))

for idx, df_iter in enumerate(dfs_gpr_train):
    plt.subplot(nb_plts, 1, idx + 1)
    df_input_iter = df_iter.drop(columns = dict_cols['y'][1] + dict_cols['u'][1])
    df_output_iter = df_iter[dict_cols['y'][1]]
    np_input_iter = df_input_iter.to_numpy()
    np_output_iter = df_output_iter.to_numpy().reshape(-1, 1)
    
    mean, var = m.predict_f(np_input_iter)
    
    plt.plot(df_iter.index, np_output_iter[:, :], label = 'Measured data')
    plt.plot(df_iter.index, mean[:, :], label = 'Gaussian Process Prediction')
    plt.fill_between(
        df_iter.index, 
        mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
        mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
        alpha = 0.2
    )
    plt.title(f"Model Performance on training data: {train_exps[idx]}")
    plt.legend()
plt.show()

## Evaluate performance on test data

In [ ]:
dfs_gpr_test = []
for df_sc in dfs_test_sc:
    dfs_gpr_test.append(data_to_gpr(df_sc, dict_cols))

In [ ]:
plt.figure(figsize = (20, 20))

for idx, df_iter in enumerate(dfs_gpr_test):
    plt.subplot(nb_plts, 1, idx + 1)
    df_input_iter = df_iter.drop(columns = dict_cols['y'][1] + dict_cols['u'][1])
    df_output_iter = df_iter[dict_cols['y'][1]]
    np_input_iter = df_input_iter.to_numpy()
    np_output_iter = df_output_iter.to_numpy().reshape(-1, 1)
    
    mean, var = m.predict_f(np_input_iter)
    
    plt.plot(df_iter.index, np_output_iter[:, :], label = 'Measured data')
    plt.plot(df_iter.index, mean[:, :], label = 'Gaussian Process Prediction')
    plt.fill_between(
        df_iter.index, 
        mean[:, 0] - 1.96 * np.sqrt(var[:, 0]),
        mean[:, 0] + 1.96 * np.sqrt(var[:, 0]),
        alpha = 0.2
    )
    plt.title(f"Model Performance on test data: {test_exps[idx]}")
    plt.legend()
plt.show()